# Data Scraping Notebook

This notebook takes the data in the table from <a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M">this Wikipedia page</a> and turns it into a dataframe in order to work with it as needed. The necessary filtering is also done before the data is written into a file.

### Importing libraries needed 

In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

### Reading the html content from web and filtering the information from table

In [2]:
html = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
html_code = BeautifulSoup(html, 'html.parser')
table = html_code.find('table') # Looking into the table content in the html code

### Extracting information from the table and making it into a dataframe

In [3]:
data = []
for rowlevel in table.find_all('tr')[1:]: # Not looking for 0 as it contains headers in the html
    row_data = rowlevel.find_all('td')
    data.append([info.text if '\n' not in info.text else info.text.split('\n')[0] for info in row_data]) # Removing line ender '\n' from the text
df = pd.DataFrame(data, columns = ['PostalCode', 'Borough', 'Neighborhood'])
df.head(5)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Filtering the data as needed

#### 1. Removing rows where Borough is Not assigned

In [4]:
df[df['Borough']=='Not assigned'].reset_index(drop=True).head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M8A,Not assigned,
3,M2B,Not assigned,
4,M7B,Not assigned,


Seeing that the data consists of 'Not assigned' in the Borough column, it needs to be filtered out

In [5]:
df=df[df['Borough']!='Not assigned'].reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


#### 2. Grouping all Neighborhoods under each postal code in format required 

In [6]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [7]:
df.groupby(['PostalCode','Borough'],as_index=False)['Neighborhood'].count().sort_values('Neighborhood',ascending=False).head(3)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,1
65,M5R,Central Toronto,1
75,M6G,Downtown Toronto,1


We can see from the first print of head that there are multiple neighborhoods placed in a single row, this has to be corrected in a way that their seperator which is currently '/' should be ','

From the second print we can see that there are no rows which contain information for multiple neighborhoods in each borough

Hence, only action to be taken here is the change of seperators

In [8]:
df.Neighborhood=df.Neighborhood.str.replace(' /',',')
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### 3. Checking for a cell in Neighborhood having 'Not assigned' value and replacing it with information in Borough 

In [9]:
df[df.Borough=='Not assigned']

,PostalCode,Borough,Neighborhood


In [10]:
df[df.Borough=='']

,PostalCode,Borough,Neighborhood


Since there is no 'Not assigned' or an empyt value not doing any changes to the df

In case such a case arise, we can use np.where function to replace values as needed

### Final data size check 

In [11]:
df.shape

(103, 3)

In [12]:
df.to_excel('Toronto_neighborhood_info.xlsx') #Not saving as csv as there is a ',' value in Neighborhood column